# DNN : `Spam` dataset - version 0

## Packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from tensorflow.keras import models, layers, callbacks

2025-01-20 15:12:40.317357: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-20 15:12:40.338401: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737385960.362403   18599 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737385960.369725   18599 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-20 15:12:40.395855: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

## 1. Data

### 1.1. Import

In [2]:
# fold = '/Users/vincentlefieux/Dropbox/DocsACADEMIQUE/Data/'
# fold ="../data/" # Sur Onyxia et Google collab

# data = pd.read_csv(fold + 'spam.csv', header=0, sep=',', decimal='.')

url = 'https://archive.ics.uci.edu/static/public/94/data.csv'
data = pd.read_csv(url, header=0, sep=',', decimal='.')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4601 entries, 0 to 4600
Data columns (total 58 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   word_freq_make              4601 non-null   float64
 1   word_freq_address           4601 non-null   float64
 2   word_freq_all               4601 non-null   float64
 3   word_freq_3d                4601 non-null   float64
 4   word_freq_our               4601 non-null   float64
 5   word_freq_over              4601 non-null   float64
 6   word_freq_remove            4601 non-null   float64
 7   word_freq_internet          4601 non-null   float64
 8   word_freq_order             4601 non-null   float64
 9   word_freq_mail              4601 non-null   float64
 10  word_freq_receive           4601 non-null   float64
 11  word_freq_will              4601 non-null   float64
 12  word_freq_people            4601 non-null   float64
 13  word_freq_report            4601 

In [3]:
y = data['Class']
x = data.drop('Class', axis=1)

### 1.2. Train / validation / test split

In [4]:
x_train_valid, x_test, y_train_valid, y_test = train_test_split(x, y, test_size=1/5)

x_train, x_valid, y_train, y_valid = train_test_split(x_train_valid, y_train_valid, test_size=1/5)

In [5]:
print('Dimensions of x_train :', x_train.shape)
print('Dimensions of x_valid :', x_valid.shape)
print('Dimensions of x_test  :', x_test.shape)
print('Dimensions of y_train :', y_train.shape)
print('Dimensions of y_valid :', y_valid.shape)
print('Dimensions of y_test  :', y_test.shape)

Dimensions of x_train : (2944, 57)
Dimensions of x_valid : (736, 57)
Dimensions of x_test  : (921, 57)
Dimensions of y_train : (2944,)
Dimensions of y_valid : (736,)
Dimensions of y_test  : (921,)


### 1.3. Normalization

In [6]:
scaler = StandardScaler()
scaler.fit(x_train)

x_train_norm = scaler.transform(x_train)
x_valid_norm = scaler.transform(x_valid)
x_test_norm  = scaler.transform(x_test)

## 2. DNN

### 2.1. Architecture

In [7]:
dim_inputs  = x_test_norm.shape[1]
dim_outputs = 1

neur_layer = 20

model = models.Sequential(name='DNN')

model.add(layers.Input((dim_inputs,), name='Inputs'))
model.add(layers.Dense(neur_layer, activation='relu', name='Hidden_layer'))
model.add(layers.Dense(dim_outputs, activation='sigmoid', name='Output_layer'))

I0000 00:00:1737385967.569053   18599 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13282 MB memory:  -> device: 0, name: NVIDIA A2, pci bus id: 0000:b1:00.0, compute capability: 8.6


In [8]:
model.summary()

Model: "DNN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Hidden_layer (Dense)            │ (None, 20)             │         1,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output_layer (Dense)            │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,181 (4.61 KB)

 Trainable params: 1,181 (4.61 KB)

 Non-trainable params: 0 (0.00 B)

### 2.2. Optimization

In [9]:
model.compile(optimizer = 'adam',
              loss      = 'binary_crossentropy',
              metrics   = ['accuracy'])

In [10]:
# Entraînement avec validation
history = model.fit(x_train_norm, y_train,
                    validation_data=(x_valid_norm, y_valid), # Données de validation
                    epochs=50,  # Nombre maximal d'epochs
                    batch_size=32,  # Taille du lot
                    verbose=1)

Epoch 1/50


I0000 00:00:1737385969.685284   18973 service.cc:148] XLA service 0x7f9ba8003710 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1737385969.685348   18973 service.cc:156]   StreamExecutor device (0): NVIDIA A2, Compute Capability 8.6
2025-01-20 15:12:49.712746: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
E0000 00:00:1737385969.836722   18973 cuda_dnn.cc:522] Loaded runtime CuDNN library: 9.1.0 but source was compiled with: 9.3.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.
E0000 00:00:1737385969.889370   18973 cuda_dnn.cc:522] Loaded runtime CuDNN library: 9.1.0 but source was compiled with: 

FailedPreconditionError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/onyxia/work/.venv/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/onyxia/work/.venv/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/onyxia/work/.venv/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/onyxia/work/.venv/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/home/onyxia/work/.venv/lib/python3.10/site-packages/ipyke

In [12]:
import tensorflow as tf
print("GPUs disponibles :", tf.config.list_physical_devices('GPU'))


GPUs disponibles : [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
